source : https://github.com/IIGROUP/MANIQA/blob/master/models/maniqa.py

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Charger et prétraiter l'image
path = "chemin_vers_votre_image.jpg"
img = Image.open(path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

img = transform(img)
img = img.unsqueeze(0)  # Ajouter une dimension de lot (batch dimension)

# Charger le modèle
model = MANIQA()
model = model.cuda()  # Si le modèle est sur GPU

# Mettre le modèle en mode évaluation
model.eval()

# Passer l'image à travers le modèle
with torch.no_grad():
    img = img.cuda()  # Si l'image est sur GPU
    output = model(img)

# Le score pour l'image donnée
print("Score:", output.item())
